<a href="https://colab.research.google.com/github/RiteshDKgpian/X-tra-Telegram/blob/master/KGPOpenIITDA_NLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.analyticsvidhya.com/blog/2021/12/text-classification-using-bert-and-tensorflow/#:~:text=What%20is%20BERT%3F,and%20a%20self%2Dattention%20layer


https://towardsdatascience.com/siamese-and-dual-bert-for-multi-text-classification-c6552d435533
https://github.com/cerlymarco/MEDIUM_NoteBook/tree/master/Siamese_Dual_BERT

In [ ]:
import pandas as pd

df = pd.read_csv('classified.csv')
df.head()

,Unnamed: 0,index,text,clean_text,label
0,0,501,"Every person has a right to profess, practice ...",person profess practice propagate religion he ...,1
1,1,1784,"That is , the pigment particles mix with the v...",pigment particle mix vehicle particle distribu...,0
2,2,1625,The Prime Minister of India is the leader of t...,prime minister india leader ruling party lok s...,1
3,3,661,The water you drink now-a-days is not safe .,water drink nowadays safe,1
4,4,1675,"I , along with my mother , brothers and sister...",mother brother sister brought small bamboo,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'],df['label'], stratify=df['label'])
X_train.head(4)

307     Issue of currency notes In India , Reserve Ban...
1005    In these circumstances , the farmers can be sa...
232     If the universal soul fills the whole world, i...
797     Calculate the initial velocity if he was decel...
Name: text, dtype: object

In [ ]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 31.1 MB/s 
     |████████████████████████████████| 578.0 MB 18 kB/s 
     |████████████████████████████████| 438 kB 72.8 MB/s 
     |████████████████████████████████| 1.7 MB 57.9 MB/s 
     |████████████████████████████████| 5.9 MB 59.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model1 = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model1.fit(X_train, y_train, epochs=10, batch_size = 10)

Epoch 1/10
79/79 [==============================] - 10s 95ms/step - loss: 0.6351 - accuracy: 0.6981
Epoch 2/10
79/79 [==============================] - 7s 92ms/step - loss: 0.6236 - accuracy: 0.7070
Epoch 3/10
79/79 [==============================] - 8s 98ms/step - loss: 0.6158 - accuracy: 0.6955
Epoch 4/10
79/79 [==============================] - 7s 93ms/step - loss: 0.6275 - accuracy: 0.6917
Epoch 5/10
79/79 [==============================] - 7s 93ms/step - loss: 0.6175 - accuracy: 0.7032
Epoch 6/10
79/79 [==============================] - 7s 93ms/step - loss: 0.6043 - accuracy: 0.7121
Epoch 7/10
79/79 [==============================] - 7s 93ms/step - loss: 0.6114 - accuracy: 0.7070
Epoch 8/10
79/79 [==============================] - 7s 94ms/step - loss: 0.6175 - accuracy: 0.7006
Epoch 9/10
79/79 [==============================] - 8s 100ms/step - loss: 0.6183 - accuracy: 0.6917
Epoch 10/10
79/79 [==============================] - 7s 94ms/step - loss: 0.6235 - accuracy: 0.6803


In [ ]:
y_predicted = model1.predict(X_test)
#y_predicted = y_predicted.flatten()

9/9 [==============================] - 4s 311ms/step


In [ ]:
ypred = []

def threshold(i):
  if i >= 0.5:
    return 1
  else:
    return 0

for i in y_predicted:
  ypred.append(threshold(i))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, ypred)

array([[  0,  77],
       [  0, 185]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, ypred)

0.7061068702290076